# Dataset Index Generation 

This was originally used to generate the indices for the training, validation, and test sets for e's and gammas. 
Modified to generate indices for e's and mu's.

In [1]:
import sys
import os
import h5py
from collections import Counter
from progressbar import *
import re
import numpy as np
from scipy import signal
import matplotlib
#from watchmal.testing.repeating_classifier_training_utils import *
from functools import reduce
from IPython.display import HTML

# Add the path to the parent directory to augment search for module
par_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

print("parent directory: ", par_dir)

if par_dir not in sys.path:
    sys.path.append(par_dir)

sys.path.append("../..")
sys.path.append("..")

%load_ext autoreload
%matplotlib inline
%autoreload 2

parent directory:  /project/6008045/jgao/WatChMaL_analysis


In [3]:
original_data_path = "/scratch/jgao/data/HKHybrid/HKHybrid_e-mu-_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_6Mevts.hdf5"
f = h5py.File(original_data_path, "r")
labels = np.array(f['labels'])
print("keys: ", f.keys())
print("labels: ", labels, len(labels))

keys:  <KeysViewHDF5 ['angles', 'energies', 'event_hits_index_20', 'event_hits_index_3', 'event_ids', 'hit_charge_20', 'hit_charge_3', 'hit_pmt_20', 'hit_pmt_3', 'hit_time_20', 'hit_time_3', 'labels', 'positions', 'root_files', 'veto', 'veto2']>
labels:  [1 1 1 ... 2 2 2] 5985000


In [24]:
# Check which bustard is missed out in the 1000 mu files
segmented_data_path = "/scratch/jgao/data/HKHybrid/HKHybrid_e-mu-_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_6Mevts.hdf5"
f = h5py.File(segmented_data_path, "r")
label = np.array(f['labels'])
root_file = np.array(f['root_files'])

In [26]:
indices = np.array(range(len(label)))
mu_indices       = indices[np.where(label == 2)]
mu_root_file_set = list(dict.fromkeys(root_file[mu_indices]))
# print(mu_root_file_set)

# actual_file_list = []
# for i in range(90):
#     actual_file_list.append(root_file[i*3000])
# # print(actual_file_list)


[b'/project/rpp-blairt2k/machine_learning/data/HKHybrid/WCSim/mu-/E0to1000MeV/unif-pos-R3240-y3287cm/4pi-dir/HKHybrid_mu-_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_3000evts_0.root', b'/project/rpp-blairt2k/machine_learning/data/HKHybrid/WCSim/mu-/E0to1000MeV/unif-pos-R3240-y3287cm/4pi-dir/HKHybrid_mu-_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_3000evts_1.root', b'/project/rpp-blairt2k/machine_learning/data/HKHybrid/WCSim/mu-/E0to1000MeV/unif-pos-R3240-y3287cm/4pi-dir/HKHybrid_mu-_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_3000evts_2.root', b'/project/rpp-blairt2k/machine_learning/data/HKHybrid/WCSim/mu-/E0to1000MeV/unif-pos-R3240-y3287cm/4pi-dir/HKHybrid_mu-_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_3000evts_3.root', b'/project/rpp-blairt2k/machine_learning/data/HKHybrid/WCSim/mu-/E0to1000MeV/unif-pos-R3240-y3287cm/4pi-dir/HKHybrid_mu-_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_3000evts_4.root', b'/project/rpp-blairt2k/machine_learning/data/HKHybrid/WCSim/mu-/E0to1000MeV/unif-pos-R32

In [31]:
actual_file_list2 = [str(i).split('evts_')[-1].split('.root')[0] for i in mu_root_file_set]
actual_file_list2 = [int(i) for i in actual_file_list2]
actual_file_list2.sort()

# print(actual_file_list2)

for i in np.arange(len(actual_file_list2)-1):
    if actual_file_list2[i] + 1 != actual_file_list2[i+1]:
        print("before: ", actual_file_list2[i], " after: ", actual_file_list2[i+1])

before:  351  after:  353
before:  727  after:  730
before:  776  after:  778
before:  781  after:  783


## Load the Dataset

In [4]:
# The following are all digitised
hdf5_hit_pmt_20 = f["hit_pmt_20"]
hdf5_hit_time_20 = f["hit_time_20"]
hdf5_hit_charge_20 = f["hit_charge_20"]
hdf5_hit_pmt_3 = f["hit_pmt_3"]
hdf5_hit_time_3 = f["hit_time_3"]
hdf5_hit_charge_3 = f["hit_charge_3"]

# NO METHOD GET_OFFSET??
hit_pmt_20 = np.memmap(original_data_path, mode="r", shape=hdf5_hit_pmt_20.shape,
                       offset=hdf5_hit_pmt_20.id.get_offset(), dtype=hdf5_hit_pmt_20.dtype)
hit_time_20 = np.memmap(original_data_path, mode="r", shape=hdf5_hit_time_20.shape,
                        offset=hdf5_hit_time_20.id.get_offset(), dtype=hdf5_hit_time_20.dtype)
hit_charge_20 = np.memmap(original_data_path, mode="r", shape=hdf5_hit_charge_20.shape,
                          offset=hdf5_hit_charge_20.id.get_offset(), dtype=hdf5_hit_charge_20.dtype)
hit_pmt_3 = np.memmap(original_data_path, mode="r", shape=hdf5_hit_pmt_3.shape,
                       offset=hdf5_hit_pmt_3.id.get_offset(), dtype=hdf5_hit_pmt_3.dtype)
hit_time_3 = np.memmap(original_data_path, mode="r", shape=hdf5_hit_time_3.shape,
                        offset=hdf5_hit_time_3.id.get_offset(), dtype=hdf5_hit_time_3.dtype)
hit_charge_3 = np.memmap(original_data_path, mode="r", shape=hdf5_hit_charge_3.shape,
                          offset=hdf5_hit_charge_3.id.get_offset(), dtype=hdf5_hit_charge_3.dtype)

angles     = np.array(f['angles'])
energies   = np.array(f['energies'])
positions  = np.array(f['positions'])
labels     = np.array(f['labels'])
root_files = np.array(f['root_files'])

#original_radius = [np.sqrt(original_positions[i,0,0]**2 + original_positions[i,0,2]**2) 
#for i in range(original_positions.shape[0])]

In [12]:
for i in range(4000,4005):
    print(root_files[i])
    

b'/project/rpp-blairt2k/machine_learning/data/HKHybrid/WCSim/e-/E0to1000MeV/unif-pos-R3240-y3287cm/4pi-dir/HKHybrid_e-_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_3000evts_1.root'
b'/project/rpp-blairt2k/machine_learning/data/HKHybrid/WCSim/e-/E0to1000MeV/unif-pos-R3240-y3287cm/4pi-dir/HKHybrid_e-_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_3000evts_1.root'
b'/project/rpp-blairt2k/machine_learning/data/HKHybrid/WCSim/e-/E0to1000MeV/unif-pos-R3240-y3287cm/4pi-dir/HKHybrid_e-_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_3000evts_1.root'
b'/project/rpp-blairt2k/machine_learning/data/HKHybrid/WCSim/e-/E0to1000MeV/unif-pos-R3240-y3287cm/4pi-dir/HKHybrid_e-_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_3000evts_1.root'
b'/project/rpp-blairt2k/machine_learning/data/HKHybrid/WCSim/e-/E0to1000MeV/unif-pos-R3240-y3287cm/4pi-dir/HKHybrid_e-_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_3000evts_1.root'


In [5]:
# Set up indices
indices = np.array(range(len(labels)))

# Set up dict of file indices
file_dict = dict.fromkeys(root_files)
print("Dict set")

for file in file_dict.keys():
    file_dict[file] = []

for idx, root_file in enumerate(root_files):
    file_dict[root_file].append(idx)
print("Done")

Dict set
Done


In [14]:
# Get files associated with each particle type
gamma_indices = indices[np.where(labels == 0)]
gamma_root_file_set = list(dict.fromkeys(root_files[gamma_indices]))

e_indices     = indices[np.where(labels == 1)]
e_root_file_set = list(dict.fromkeys(root_files[e_indices]))

mu_indices    = indices[np.where(labels == 2)]
mu_root_file_set = list(dict.fromkeys(root_files[mu_indices]))

print(len(gamma_root_file_set))
print(len(e_root_file_set))
print(len(mu_root_file_set))
print(mu_root_file_set,"\n")

0
1000
995
[b'/project/rpp-blairt2k/machine_learning/data/HKHybrid/WCSim/mu-/E0to1000MeV/unif-pos-R3240-y3287cm/4pi-dir/HKHybrid_mu-_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_3000evts_0.root', b'/project/rpp-blairt2k/machine_learning/data/HKHybrid/WCSim/mu-/E0to1000MeV/unif-pos-R3240-y3287cm/4pi-dir/HKHybrid_mu-_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_3000evts_1.root', b'/project/rpp-blairt2k/machine_learning/data/HKHybrid/WCSim/mu-/E0to1000MeV/unif-pos-R3240-y3287cm/4pi-dir/HKHybrid_mu-_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_3000evts_2.root', b'/project/rpp-blairt2k/machine_learning/data/HKHybrid/WCSim/mu-/E0to1000MeV/unif-pos-R3240-y3287cm/4pi-dir/HKHybrid_mu-_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_3000evts_3.root', b'/project/rpp-blairt2k/machine_learning/data/HKHybrid/WCSim/mu-/E0to1000MeV/unif-pos-R3240-y3287cm/4pi-dir/HKHybrid_mu-_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_3000evts_4.root', b'/project/rpp-blairt2k/machine_learning/data/HKHybrid/WCSim/mu-/E0to1000MeV/u

In [15]:
# Define indices retrieval function
def get_indices_for_files(file_names):
    all_indices = []
    for file_name in file_names:
        all_indices.extend(file_dict[file_name])
    return np.array(all_indices)

In [16]:
mu_test_files, mu_val_files, mu_train_files = mu_root_file_set[0:400], mu_root_file_set[400:500], mu_root_file_set[500:]

mu_test_set, mu_val_set, mu_train_set = get_indices_for_files(mu_test_files), get_indices_for_files(mu_val_files), get_indices_for_files(mu_train_files)

print(mu_test_set)

[3000000 3000001 3000002 ... 4199997 4199998 4199999]


In [9]:
gamma_test_files, gamma_val_files, gamma_train_files = gamma_root_file_set[0:400], gamma_root_file_set[400:500], gamma_root_file_set[500:1000]

gamma_test_set, gamma_val_set, gamma_train_set = get_indices_for_files(gamma_test_files), get_indices_for_files(gamma_val_files), get_indices_for_files(gamma_train_files)

print(gamma_test_set)

[2944634 2944635 2944636 ... 4127008 4127009 4127010]


In [17]:
e_test_files, e_val_files, e_train_files = e_root_file_set[0:400], e_root_file_set[400:500], e_root_file_set[500:1000]

e_test_set, e_val_set, e_train_set = get_indices_for_files(e_test_files), get_indices_for_files(e_val_files), get_indices_for_files(e_train_files)

print(e_test_set)

[      0       1       2 ... 1199997 1199998 1199999]


In [19]:
print(len(e_train_files))
print(len(mu_train_files))

500
495


In [21]:
# Verify that indices match
all_e_indices = np.concatenate((e_test_set, e_val_set, e_train_set))
print(set(labels[all_e_indices]))

# all_gamma_indices = np.concatenate((gamma_test_set, gamma_val_set, gamma_train_set))
# print(set(labels[all_gamma_indices]))

all_mu_indices = np.concatenate((mu_test_set, mu_val_set, mu_train_set))
print(set(labels[all_mu_indices]))

{1}
{2}


In [22]:
# Verify that all events are uniquely accounted for
# all_collected_indices = np.concatenate((e_test_set, e_val_set, e_train_set, gamma_test_set, gamma_val_set, gamma_train_set))
all_collected_indices = np.concatenate((e_test_set, e_val_set, e_train_set, mu_test_set, mu_val_set, mu_train_set))

print(len(labels))
print(len(all_collected_indices))
print(len(set(all_collected_indices)))

5985000
5985000
5985000


In [23]:
train_idxs = np.concatenate((e_train_set, mu_train_set))
val_idxs   = np.concatenate((e_val_set, mu_val_set))
test_idxs  = np.concatenate((e_test_set, mu_test_set))

In [24]:
np.savez('/scratch/jgao/HKHybrid_e-mu-_E0to1000MeV_unif-pos-R3240-y3287cm_4pi-dir_idxs.npz', 
         train_idxs=train_idxs, val_idxs=val_idxs, test_idxs=test_idxs)